In [1]:
import cv2
import numpy as np
import os
from random import shuffle

import tensorflow as tf

C:\Users\Fernando\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
TRAIN_DIR = './train' 
TEST_DIR = './test'

IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = "dogsvscats-{}-{}.model".format(LR, '2conv-basic-new-bigger')
TRAIN_DATA_FILE = 'sentdex.training.npy'
TEST_DATA_FILE = 'sentdex.test.npy'

In [3]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat' : return [1,0]
    else : return [0,1]


In [4]:
def create_train_data():
    training_data = []
    for img in os.listdir(TRAIN_DIR) :
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
        
    shuffle(training_data)
    np.save(TRAIN_DATA_FILE, training_data)
    return training_data

In [5]:
def process_test_data():
    testing_data = []
    for img in os.listdir(TEST_DIR):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    shuffle(testing_data)
    np.save(TEST_DATA_FILE, testing_data)
    return testing_data


In [6]:
#train_data = create_train_data()

In [7]:
train_data = np.load(TRAIN_DATA_FILE)

In [8]:
#def model_fn(features, labels, mode):
def model_fn() :
    """Model function for CNN."""
    # Input Layer
    
    
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(32,5,activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,1)))
    #model.add(tf.keras.layers.MaxPool2D(5))
    
    model.add(tf.keras.layers.Conv2D(64,5,activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D(5))
    
    
    model.add(tf.keras.layers.Conv2D(128,5,activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D(5))
    
    
    model.add(tf.keras.layers.Conv2D(64,5,activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D(5))
    
    
    model.add(tf.keras.layers.Conv2D(32,5,activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D(5))
    
    model.add(tf.keras.layers.Flatten())
    
    model.add(tf.keras.layers.Dense(1024, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.25))
    
    model.add(tf.keras.layers.Dense(2, activation='softmax'))
    
    
    return model

In [9]:
train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = np.array([i[1] for i in train])

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = np.array([i[1] for i in test])


In [10]:
model = model_fn()

model.compile(
    loss=tf.keras.losses.mean_squared_error,
    optimizer=tf.keras.optimizers.Adadelta(LR),
    metrics=['accuracy']
)

In [18]:
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='./logs' + MODEL_NAME, histogram_freq=0,
                          write_graph=True, write_images=False)

model.fit(
    X, #inputs
    Y, #features
    epochs=10, 
    batch_size=4,
    validation_data=(test_x,test_y),
    shuffle=True,
    callbacks=[tensorboard]
)

Train on 24500 samples, validate on 500 samples
Epoch 1/10
24500/24500 [==============================] - 510s 21ms/step - loss: 0.0863 - acc: 0.8990 - val_loss: 0.1903 - val_acc: 0.7280
Epoch 2/10
24500/24500 [==============================] - 510s 21ms/step - loss: 0.0832 - acc: 0.9012 - val_loss: 0.1928 - val_acc: 0.7180
Epoch 3/10
24500/24500 [==============================] - 508s 21ms/step - loss: 0.0786 - acc: 0.9091 - val_loss: 0.1921 - val_acc: 0.7320
Epoch 4/10
24500/24500 [==============================] - 515s 21ms/step - loss: 0.0755 - acc: 0.9136 - val_loss: 0.1896 - val_acc: 0.7440
Epoch 5/10
24500/24500 [==============================] - 509s 21ms/step - loss: 0.0716 - acc: 0.9192 - val_loss: 0.1925 - val_acc: 0.7320
Epoch 6/10
24500/24500 [==============================] - 510s 21ms/step - loss: 0.0679 - acc: 0.9247 - val_loss: 0.1912 - val_acc: 0.7280
Epoch 7/10
24500/24500 [==============================] - 510s 21ms/step - loss: 0.0645 - acc: 0.9301 - val_loss: 0.18

In [19]:
model.save(MODEL_NAME)
#model = tf.keras.models.load_model(MODEL_NAME)

In [28]:
test_data_label = process_test_data()
test_data = np.array([i[0] for i in test_data_label]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_label = [i[1] for i in test_data_label]

In [29]:
prediction = model.predict(test_data)

In [36]:
template = '{},{}\n'
submission_file = 'submission_file_keras-bigger.csv'
with open(submission_file,'w') as f:
    f.write('id,label\n')
            
with open(submission_file,'a') as f:
    for label, pred in zip(test_label, prediction) :
        f.write(template.format(label, pred[1]))

In [14]:
import matplotlib.pyplot as plt

#fig = plt.figure()
im = test_x[0]
plt.imshow(im.reshape(im.shape[0], im.shape[1]))

In [15]:
eval_x = np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = np.array([i[1] for i in test])